# Segmentation

Segments using a histogram based approach, trims the segmentation region to only consist of the tumour, then saves as .img.gz file.

NOTE: if changing the segmentation filename, make changes to the segment_row function.

In [1]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg pandarallel

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

from tqdm import tqdm

from pandarallel import pandarallel

import re

tqdm.pandas()
pandarallel.initialize(progress_bar=True,nb_workers= 40)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
annotation_boxes = pd.read_csv("../Data/segmentation_annotations_NIFTI.csv")#.set_index("Patient_ID")
DATASET_PATH = '../../Processed NIFTI Dataset/'

In [4]:
annotation_boxes

,Patient_ID,Start Row,End Row,Start Column,End Column,Start Slice,End Slice
0,Breast_MRI_001,234,271,308,341,89,112
1,Breast_MRI_002,251,294,108,136,59,72
2,Breast_MRI_003,351,412,82,139,96,108
3,Breast_MRI_004,262,280,193,204,86,95
4,Breast_MRI_005,188,213,138,178,76,122
...,...,...,...,...,...,...,...
917,Breast_MRI_918,345,395,338,395,62,85
918,Breast_MRI_919,285,312,369,397,98,109
919,Breast_MRI_920,172,193,337,355,87,101
920,Breast_MRI_921,328,374,404,446,97,121


In [5]:
filenames = ['pre.img.gz', 'post_1.img.gz']

In [6]:
# generate segmentation masks for each patient in the directory

def segment_row(x):
    import SimpleITK as sitk
    import os
    import numpy as np

    filenames = ['pre.img.gz', 'post_1.img.gz']
    DATASET_PATH = '../../Processed NIFTI Dataset/'
    SEGMENTATION_FILENAME = 'segmentation.img.gz'


    patient_id = x['Patient_ID']
    patient_path = os.path.join(DATASET_PATH, patient_id)

    row1 = x['Start Row']
    row2 = x['End Row']

    col1 = x['Start Column']
    col2 = x['End Column']

    slice1 = x['Start Slice']
    slice2 = x['End Slice']

    if(os.path.isdir(patient_path)):
        # find files with paths here
        img_path = os.path.join(patient_path, filenames[1])   #load pre path and generate segments from this
        img = sitk.ReadImage(img_path)

        
        #filter instantiation : Otsu filtering
        otsu_filter = sitk.OtsuThresholdImageFilter()
        otsu_filter.SetInsideValue(0)
        otsu_filter.SetOutsideValue(1)

        # segmentation and getting back array
        seg = otsu_filter.Execute(img)
        seg = sitk.GetArrayFromImage(seg)

        #mask generation
        mask = np.zeros(seg.shape)
        mask[slice1:slice2, row1:row2, col1:col2] = seg[slice1:slice2, row1:row2, col1:col2]

        segment = sitk.GetImageFromArray(mask)
        segment.CopyInformation(img)

        sitk.WriteImage(segment, os.path.join(patient_path, SEGMENTATION_FILENAME))

    
annotation_boxes.parallel_apply(segment_row, axis = 1)


0      None
1      None
2      None
3      None
4      None
       ... 
917    None
918    None
919    None
920    None
921    None
Length: 922, dtype: object